![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

# Getting Started with Iceberg and Spark

In [98]:
spark

## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [99]:
%%sql

DROP TABLE IF EXISTS taxis

++
||
++
++

In [100]:
df = spark.read.option("header", True).option("inferSchema", "true").csv("/home/iceberg/data/yello_tripdata_2020-04.csv")
df.write.saveAsTable("taxis")

In [101]:
%%sql

DESCRIBE EXTENDED taxis

col_name,data_type,comment
VendorID,int,
tpep_pickup_datetime,string,
tpep_dropoff_datetime,string,
passenger_count,int,
trip_distance,double,
RatecodeID,int,
store_and_fwd_flag,string,
PULocationID,int,
DOLocationID,int,
payment_type,int,


In [102]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
237993


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [103]:
%%sql

ALTER TABLE taxis RENAME COLUMN fare_amount TO fare

++
||
++
++

In [104]:
%%sql

ALTER TABLE taxis RENAME COLUMN trip_distance TO distance

++
||
++
++

In [105]:
%%sql

ALTER TABLE taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

++
||
++
++

In [106]:
%%sql

ALTER TABLE taxis ALTER COLUMN distance TYPE double;

++
||
++
++

In [107]:
%%sql

ALTER TABLE taxis ALTER COLUMN distance AFTER fare;

++
||
++
++

In [108]:
%%sql

ALTER TABLE taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

++
||
++
++

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [109]:
%%sql

UPDATE taxis
SET fare_per_distance_unit = fare/distance

++
||
++
++

In [110]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2020-04-01 00:41:22,2020-04-01 01:01:53,5.5,1.2,4.583333492279053
1,2020-04-01 00:56:00,2020-04-01 01:09:25,12.5,3.4,3.6764705181121826
1,2020-04-01 00:00:26,2020-04-01 00:09:25,10.0,2.8,3.5714285373687744
1,2020-04-01 00:24:38,2020-04-01 00:34:38,10.0,2.6,3.846153736114502
2,2020-04-01 00:13:24,2020-04-01 00:18:26,6.5,1.44,4.513888835906982
2,2020-04-01 00:24:36,2020-04-01 00:33:09,10.5,2.93,3.583617687225342
2,2020-04-01 00:56:56,2020-04-01 01:09:13,20.0,6.86,2.915452003479004
2,2020-04-01 00:06:56,2020-04-01 00:14:15,7.0,1.19,5.882352828979492
1,2020-04-01 00:50:05,2020-04-01 01:08:54,31.5,11.3,2.7876105308532715
2,2020-04-01 00:07:10,2020-04-01 00:18:45,13.0,3.68,3.5326087474823


## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 and a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [111]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

++
||
++
++

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [112]:
%%sql

DELETE FROM taxis
WHERE fare_per_distance_unit is null

++
||
++
++

In [113]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
2,2020-04-01 00:42:56,2020-04-01 00:48:41,-7.0,1.34,-5.223880767822266
2,2020-04-01 00:26:49,2020-04-01 00:31:04,6.5,1.69,3.846153736114502
2,2020-04-01 00:42:30,2020-04-01 00:46:58,6.5,1.7,3.8235294818878174
1,2020-04-01 00:36:15,2020-04-01 00:41:30,7.0,1.8,3.8888888359069824
2,2020-04-01 01:16:34,2020-04-01 01:23:20,-8.0,1.85,-4.324324131011963
2,2020-04-01 01:51:36,2020-04-01 01:57:59,7.5,1.88,3.9893617630004883
1,2020-04-01 02:13:27,2020-04-01 02:16:30,6.0,1.5,4.0
2,2020-04-01 02:25:38,2020-04-01 02:26:05,-2.5,0.08,-31.25
2,2020-04-01 02:29:54,2020-04-01 02:33:27,6.0,1.52,3.9473683834075928
2,2020-04-01 02:39:47,2020-04-01 02:45:27,-8.0,1.94,-4.123711109161377


In [114]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
4953


## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [115]:
%%sql

ALTER TABLE taxis
ADD PARTITION FIELD VendorID

++
||
++
++

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [116]:
%%sql

SELECT snapshot_id, manifest_list
FROM taxis.snapshots

snapshot_id,manifest_list
3873268274651895103,/home/iceberg/warehouse/taxis/metadata/snap-3873268274651895103-1-445ac8c4-398c-42bb-aaf7-e7a871653908.avro
8068778203692398642,/home/iceberg/warehouse/taxis/metadata/snap-8068778203692398642-1-dec5add0-870c-4f0e-91d0-71b9b070af59.avro
6243683758127722975,/home/iceberg/warehouse/taxis/metadata/snap-6243683758127722975-1-8fd194bd-343d-497a-b49f-768b555992f3.avro
3881209843136635094,/home/iceberg/warehouse/taxis/metadata/snap-3881209843136635094-1-6626c20b-cb57-473f-9d0e-45c3e03e1506.avro


The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [117]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM taxis.files

file_path,file_format,record_count,null_value_counts,lower_bounds,upper_bounds
/home/iceberg/warehouse/taxis/data/00000-132-bf28bb91-5c0f-4d06-bcf3-fceb3bc00043-00001.parquet,PARQUET,4953,"{1: 53, 2: 0, 3: 0, 4: 53, 5: 0, 6: 53, 7: 53, 8: 0, 9: 0, 10: 53, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0}","{1: bytearray(b'\x01\x00\x00\x00'), 2: bytearray(b'2020-04-01 00:26'), 3: bytearray(b'2020-04-01 00:31'), 4: bytearray(b'\x00\x00\x00\x00'), 5: bytearray(b'{\x14\xaeG\xe1z\x84?'), 6: bytearray(b'\x01\x00\x00\x00'), 7: bytearray(b'N'), 8: bytearray(b'\x04\x00\x00\x00'), 9: bytearray(b'\x04\x00\x00\x00'), 10: bytearray(b'\x01\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00J\xc0'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x12\xc0'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0\xbf'), 14: bytearray(b'\x00\x00\x00\x00\x00\x00\x06\xc0'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 16: bytearray(b'333333\xd3\xbf'), 17: bytearray(b'fffff\xe6M\xc0'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 19: bytearray(b'\x00\x80\xa2\xc5')}","{1: bytearray(b'\x02\x00\x00\x00'), 2: bytearray(b'2020-04-30 23:27'), 3: bytearray(b'2020-04-30 23:32'), 4: bytearray(b'\x06\x00\x00\x00'), 5: bytearray(b'\x00\x00\x00\x00\x00\x00\x00@'), 6: bytearray(b'\x05\x00\x00\x00'), 7: bytearray(b'Y'), 8: bytearray(b'\t\x01\x00\x00'), 9: bytearray(b'\t\x01\x00\x00'), 10: bytearray(b'\x04\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00 @'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x0c@'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 14: bytearray(b'\n\xd7\xa3p=\xaaO@'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00N@'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'\xb8\x1e\x85\xebQ\xc8R@'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x80@')}"


## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [118]:
%%sql

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2022-01-31 05:09:20.982000,3873268274651895103,None,True
2022-01-31 05:09:23.710000,8068778203692398642,3873268274651895103,True
2022-01-31 05:09:24.800000,6243683758127722975,8068778203692398642,True
2022-01-31 05:09:25.226000,3881209843136635094,6243683758127722975,True


You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [119]:
%%sql --var df

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2022-01-31 05:09:20.982000,3873268274651895103,None,True
2022-01-31 05:09:23.710000,8068778203692398642,3873268274651895103,True
2022-01-31 05:09:24.800000,6243683758127722975,8068778203692398642,True
2022-01-31 05:09:25.226000,3881209843136635094,6243683758127722975,True


In [120]:
original_snapshot = df.head().snapshot_id
print(original_snapshot)
spark.sql(f"CALL demo.system.rollback_to_snapshot('taxis', {original_snapshot})")

3873268274651895103


DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]

In [121]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2020-04-01 00:41:22,2020-04-01 01:01:53,5.5,1.2,None
1,2020-04-01 00:56:00,2020-04-01 01:09:25,12.5,3.4,None
1,2020-04-01 00:00:26,2020-04-01 00:09:25,10.0,2.8,None
1,2020-04-01 00:24:38,2020-04-01 00:34:38,10.0,2.6,None
2,2020-04-01 00:13:24,2020-04-01 00:18:26,6.5,1.44,None
2,2020-04-01 00:24:36,2020-04-01 00:33:09,10.5,2.93,None
2,2020-04-01 00:56:56,2020-04-01 01:09:13,20.0,6.86,None
2,2020-04-01 00:06:56,2020-04-01 00:14:15,7.0,1.19,None
1,2020-04-01 00:50:05,2020-04-01 01:08:54,31.5,11.3,None
2,2020-04-01 00:07:10,2020-04-01 00:18:45,13.0,3.68,None


Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [122]:
%%sql

SELECT *
FROM taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2022-01-31 05:09:20.982000,3873268274651895103,None,True
2022-01-31 05:09:23.710000,8068778203692398642,3873268274651895103,False
2022-01-31 05:09:24.800000,6243683758127722975,8068778203692398642,False
2022-01-31 05:09:25.226000,3881209843136635094,6243683758127722975,False
2022-01-31 05:09:25.740000,3873268274651895103,None,True


In [123]:
%%sql

SELECT COUNT(*) as cnt
FROM taxis

cnt
237993
